In [4]:
#B-tree
class node:
    def __init__(self, t):
        self.t = t
        self.leaf = True
        self.keys = []
        self.children = []

class Btree(object):
    def __init__(self, t=2):
        self.t = t
        self.root = node()

 #   def insert(self, val):




In [5]:
test = Btree()

for i in range(1,9):
    test.insert(i)
test.print_order()

TypeError: node.__init__() missing 1 required positional argument: 't'